이동평균+원본

In [ ]:
def make_dataset(times,ma,data=data):
    
    select_features=['wl_js_br','wl_cd_br','wl_hj_br','wl_hg_br','wl_gj_br','wl_pd_br',
                'fw_cd_br','fw_hj_br','fw_hg_br','fw_pd_br']
    
    x = data.copy()
    x=x.set_index('ymdhm')
    data=data.set_index('ymdhm')
    data.index=pd.to_datetime(data.index)
    
    y = target.shift(-times)
    y_train = y[:497952 - times]
    y_test = y[497952 - times:]
    
    if(ma>1):
        for i in range(len(select_features)):
            coln=select_features[i]+'_lag'
            x[coln] = x[select_features[i]].rolling(window=ma).mean()
            for j in range(ma-1):
                if pd.isnull(x[coln].iloc[j]):
                    x[coln].iloc[j]=x[select_features[i]].iloc[j]
        
    x_train = x[:497952 - times]
    x_test = x[497952 - times:]
    
    
    x_train=x_train.fillna(x_train.mean())
    x_test=x_test.fillna(x_test.mean())
    
    dataset=[x_train,x_test,y_train,y_test]
    
    return dataset


시차only(원본도 하려면 drop만 빼면됨)

In [1]:
def make_datasetlagonly(times,flag,data):
    
    x = data.copy()
    x=x.set_index('ymdhm')
    data=data.set_index('ymdhm')
    data.index=pd.to_datetime(data.index)
    
    y = target.shift(-times)
    y_train = y[:497952 - times]
    y_test = y[497952 - times:]
    
    if(flag==1):
        #x["wl_cheongdam_lag"] = data['wl_cheongdam'].shift(0, freq=DateOffset(minutes=10))
        #x["fw_cheongdam_lag"] = data['fw_cheongdam'].shift(0, freq=DateOffset(minutes=10))
        #x["wl_hangang_lag"] = data['wl_hangang'].shift(0, freq=DateOffset(minutes=10))
        #x["fw_hangang_lag"] = data['fw_hangang'].shift(-9, freq=DateOffset(minutes=10))
        x["wl_haengju_lag"] = data['wl_haengju'].shift(5, freq=DateOffset(minutes=10))
        x["fw_haengju_lag"] = data['fw_haengju'].shift(5, freq=DateOffset(minutes=10))
        x["wl_paldang_lag"] = data['wl_paldang'].shift(12, freq=DateOffset(minutes=10))
        x["fw_paldang_lag"] = data['fw_paldang'].shift(15, freq=DateOffset(minutes=10))
        #x['wl_gwangjin_lag'] = data['wl_gwangjin'].shift(0, freq=DateOffset(minutes=10))
        x["inf_lag"] = data['inf'].shift(17, freq=DateOffset(minutes=10))
        x["tototf_lag"] = data['tototf'].shift(19, freq=DateOffset(minutes=10))
        x.drop(['wl_haengju','fw_haengju','wl_paldang','fw_paldang','inf','tototf'],axis=1,inplace=True)
        
    
    x_train = x[:497952 - times]
    x_test = x[497952 - times:]
    
    
    x_train=x_train.fillna('bfill')
    x_test=x_test.fillna('bfill')
    
    dataset=[x_train,x_test,y_train,y_test]
    
    return dataset


In [ ]:
# 선행시간, 이동평균, 윈도우에 맞게 데이터셋을 제공 
def load_dataset(leadtime,moving_average,window):
    
    select_features=['wl_js_br','wl_cd_br','wl_hj_br','wl_hg_br','wl_gj_br','wl_pd_br',
                'fw_cd_br','fw_hj_br','fw_hg_br','fw_pd_br','wl_pd_dam','tototf_pd_dam']
    
    fe_list=['cd_br','hj_br','hg_br','pd_br']
    
    x=pd.concat([train_data,test_data],axis=0)
    x.reset_index(drop=True,inplace=True)
    x=x.set_index('ymdhm')
    x.index=pd.to_datetime(x.index)
    
    # 선행시간을 적용하기 위해 타겟을미뤄줌 
    y=x['target'].shift(-leadtime)
    x.drop('target',axis=1,inplace=True)
            
    # 이동평균을 적용 
    if(moving_average>1):
        # 월과 시간에 대한 feature도 추가해줌 
        x['month'],x['hour']=x.index.month,x.index.hour
        for i in range(len(select_features)):
            coln=select_features[i]+str(moving_average)+'ma'
            x[coln] = x[select_features[i]].rolling(window=moving_average).mean()
    # window를 적용
    if(window>0):
        for col in select_features:
            x["{}_{}min_prev".format(col, window * 10)] = x[col].shift(window, freq=DateOffset(minutes=10))
        
    # train과 test를 다시 분리       
    idx = x.index.get_loc('2022-06-21 00:00:00')
    
    # test를 위해 train과 test의 범위를 선행시간만큼 조정 
    x_train=x[:idx-leadtime]
    x_test=x[idx-leadtime:]
    y_train = y[:idx - leadtime]

    # 이동평균혹은 window를 적용하면 생기는 nan을 없애주기 위함 
    if(moving_average>1 and window==0):
        x_train=x_train[moving_average:]
        y_train=y_train[moving_average:]
        
    if(window>0 and moving_average==0):
        x_train=x_train[window:]
        y_train=y_train[window:]
        
    dataset=[x_train,x_test,y_train]
    
    return dataset

In [ ]:
def make_dataset_windoworigin(times,windows,data):
    
    sel_feat=['wl_jamsu', 'wl_gwangjin','wl_cheongdam', 'fw_cheongdam', 
              'wl_haengju','fw_haengju','wl_hangang', 'fw_hangang',
              'wl_paldang','fw_paldang', 'swl', 'inf','tide_level']
    
    x = data.copy()
    x=x.set_index('ymdhm')
    data=data.set_index('ymdhm')
    data.index=pd.to_datetime(data.index)
    
    y = target.shift(-times)
    y_train = y[:497952 - times]
    y_test = y[497952 - times:]
    
    for col in sel_feat:
        for i in range(1, windows + 1):
            x["{}_{}prev".format(col, i)] = data[col].shift(i, freq=DateOffset(minutes=10))
        
    
    x_train = x[:497952 - times]
    x_test = x[497952 - times:]
    
    
    x_train=x_train.fillna(method='bfill')
    x_test=x_test.fillna(method='bfill')
    x_train=x_train.fillna(method='ffill')
    x_test=x_test.fillna(method='ffill')
    
    dataset=[x_train,x_test,y_train,y_test]
    
    return dataset
